In [ ]:
%pip install pandas

# Gathering data from council XLS files

In [15]:
import pandas as pd
import requests
import os

# Create list of target URLs to download
classified_urls_df = pd.read_csv('../wayback/url_classified.csv')
classified_urls_df['target_url'] = classified_urls_df['wayback_download_url'].str.split('id_/').str[1]

list_new_businesses = classified_urls_df[classified_urls_df['category'] == 'new_businesses']['target_url'].dropna().unique()
list_accounts_no_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_no_relief']['target_url'].dropna().unique()
list_account_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_relief']['target_url'].dropna().unique()
list_accouts_closed = classified_urls_df[classified_urls_df['category'] == 'accounts_closed']['target_url'].dropna().unique()
list_account_current_in_credit = classified_urls_df[classified_urls_df['category'] == 'accounts_current_in_credit']['target_url'].dropna().unique()

# Download raw files to local
save_dirs = ['new_businesses', 'accounts_no_relief', 'accounts_relief', 'accounts_closed', 'accounts_current_in_credit']

config = {
    'new_businesses': {
        'urls': list_new_businesses,
        'save_dir': 'new_businesses',
    },
    'accounts_no_relief': {
        'urls': list_accounts_no_relief,
        'save_dir': 'accounts_no_relief',
    },
    'accounts_relief': {
        'urls': list_account_relief,
        'save_dir': 'accounts_relief',
    },
    'accounts_closed': {
        'urls': list_accouts_closed,
        'save_dir': 'accounts_closed',
    },
    'accounts_current_in_credit': {
        'urls': list_account_current_in_credit,
        'save_dir': 'accounts_current_in_credit',
    },
}

for save_dir in save_dirs:
    os.makedirs('raw_files/' + save_dir, exist_ok=True)

for category, config in config.items():
    for url in config['urls']:
        raw_file_name = url.split('/')[-1]
        # Stream the content and write it to disk
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # raises exception if download failed
            with open('raw_files/' + config['save_dir'] + '/' + raw_file_name, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)


a


In [4]:
%pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
